<h1>Esse notebook tem como foco apresentar possibilidades de seleção de features com base em técnicas de seleção das bibliotecas do SKLearn e Feature Engine para construção de modelos com inúmeras variáveis</h1>

**Bibliotecas**

In [90]:
#Leitura dos dados
import pandas as pd

#Separação dos de teste e treino
from sklearn.model_selection import train_test_split

#Tratamento de dados faltantens, dummies e normalização de escala
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#Construção de pipelines de tratamento de dados
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Feature selection
from sklearn.feature_selection import VarianceThreshold
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.feature_selection import RFE

#Modelos utilizados
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

#Métricas gerais de avaliação de modelo
from sklearn.metrics import mean_squared_error, r2_score

**Informações Gerais**

In [91]:
#Leitura e visualização dos dados
df = pd.read_csv("train.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [92]:
#Informações sobre as variáveis
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [93]:
#Verificando as colunas que possuem valores faltantes
df.loc[:, df.isnull().mean()>0].isnull().mean().sort_values()

Electrical      0.000685
MasVnrArea      0.005479
BsmtQual        0.025342
BsmtCond        0.025342
BsmtFinType1    0.025342
BsmtExposure    0.026027
BsmtFinType2    0.026027
GarageCond      0.055479
GarageQual      0.055479
GarageFinish    0.055479
GarageYrBlt     0.055479
GarageType      0.055479
LotFrontage     0.177397
FireplaceQu     0.472603
MasVnrType      0.597260
Fence           0.807534
Alley           0.937671
MiscFeature     0.963014
PoolQC          0.995205
dtype: float64

**<h1> Tratamentos Iniciais </h1>**

In [94]:
'''
Criando uma variável com as colunas que contém dados vazios
Filtrando apenas as colunas as colunas que possuem dados faltantes
maiores que 5% da base e removendo-as do dataset
'''
missing_prportion = df.loc[:, df.isnull().mean()>0].isnull().mean()
cols_to_drop = missing_prportion[missing_prportion > 0.5].index
df.drop(columns=cols_to_drop, inplace=True)

In [41]:
#Selecionando as variáveis categóricas e numéricas com base no tipo da coluna
numerical_cols = df.select_dtypes(exclude='object').columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()

#Removendo as colunas SalePrice e Id das colunas númericas
numerical_cols = [col for col in numerical_cols if col not in ['SalePrice', 'Id']]
#Apenas selecionando variáveis categóricas que possuem até 3 valores distintos
categorical_cols = [col for col in categorical_cols if df[col].nunique() <= 3]

In [42]:
#Separando as features iniciais e o valor buscado
X = df[numerical_cols + categorical_cols]
y = df['SalePrice']

In [43]:
#Separando o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**<h3> Pipelines para tratamento de dados </h3>**
Nesse primeiro momento foi feito dois pipelines para tratar as variáveis númericas e categóricas.
- As variaveis númericas com valores vazios foi aplicado o valor da mediana da feature, além da normalização da escala para todas as variáveis categóricas.
- As variaveis categóricas com valores vazios foi aplicado o valor mais frequente encontrado, e a transformação das colunas textuais em dummies.

In [ ]:
#Critérios de tratamento para as colunas númericas
numeric_transformer = Pipeline(steps=[
    ('imputerNumeric', SimpleImputer(strategy='median')),
    ('standardScaler', StandardScaler())
])

#Critérios de tratamento para as colunas categóricas
categorical_transformer = Pipeline(steps=[
    ('imputerCategorical', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

#Realização do tratamento estípulados nos pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

**<h3> Pipelines com alguns tipos de tratamentos finais </h3>**
Após os tratamentos gerais feitos, nesse segundo momento é feito alguns pipelines com técnicas de seleção de features e previsões para entender quais tiverem o menor erro quadrado médio e maior R².

**<span style="color:orange">Pipeline sem tratamento adicional em um modelo de regressão de Random Forest</span>**

Nesse pipeline não foi aplicado nenhuma técnica para filtrar features.

In [147]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('randomForest', RandomForestRegressor())
])

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputerNumeric',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardScaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2nd...
                                                   'GarageYrBlt', 'GarageCars',
                                                   'GarageArea', 'WoodDeckSF',
                                                   'OpenPorchSF',
                                                   'EnclosedPorch', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputerCategorical',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Street', 'Utilities',
                                                   'LandSlope', 'CentralAir',
                                                   'GarageFinish',
                                                   'PavedDrive'])])),
                ('randomForest', RandomForestRegressor())])

In [148]:
y_pred = model_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

842632962.4154353
0.8901437905436297


**<span style="color:orange">Pipeline com variância mínima em um modelo de regressão de Random Forest</span>**

Nesse pipeline foi aplicado apenas a técnica de variância mínima para filtrar as features.

In [162]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('varianceThreshold', VarianceThreshold(0.15)),
    ('randomForest', RandomForestRegressor())
])

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputerNumeric',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardScaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2nd...
                                                   'OpenPorchSF',
                                                   'EnclosedPorch', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputerCategorical',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Street', 'Utilities',
                                                   'LandSlope', 'CentralAir',
                                                   'GarageFinish',
                                                   'PavedDrive'])])),
                ('varianceThreshold', VarianceThreshold(threshold=0.15)),
                ('randomForest', RandomForestRegressor())])

In [163]:
y_pred = model_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

833833587.0615345
0.8912909875618811


**<span style="color:orange">Pipeline com validação de correlação entre features em um modelo de regressão de Random Forest</span>**

Nesse pipeline foi aplicado apenas a técnica remover features com alta correlação entre si para remover dados redundantes.

In [108]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smatedCorr', SmartCorrelatedSelection(threshold=0.8, selection_method='variance')),
    ('randomForest', RandomForestRegressor())
])

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputerNumeric',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardScaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2nd...
                                                   'EnclosedPorch', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputerCategorical',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Street', 'Utilities',
                                                   'LandSlope', 'CentralAir',
                                                   'GarageFinish',
                                                   'PavedDrive'])])),
                ('smatedCorr',
                 SmartCorrelatedSelection(selection_method='variance')),
                ('randomForest', RandomForestRegressor())])

In [109]:
y_pred = model_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

882726593.6872934
0.884916681527811


**<span style="color:orange">Pipeline com validação de correlação entre features e variância mínima em um modelo de regressão de Random Forest</span>**

Nesse pipeline foi aplicado tanto a técnica de variância mínima, quanto a técnica remover features com alta correlação entre si.

In [170]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('varianceThreshold', VarianceThreshold(0.15)),
    ('smatedCorr', SmartCorrelatedSelection(threshold=0.8, selection_method='variance')),
    ('randomForest', RandomForestRegressor())
])

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputerNumeric',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardScaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2nd...
                                                  Pipeline(steps=[('imputerCategorical',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Street', 'Utilities',
                                                   'LandSlope', 'CentralAir',
                                                   'GarageFinish',
                                                   'PavedDrive'])])),
                ('varianceThreshold', VarianceThreshold(threshold=0.15)),
                ('smatedCorr',
                 SmartCorrelatedSelection(selection_method='variance')),
                ('randomForest', RandomForestRegressor())])

In [171]:
y_pred = model_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

834004939.0052686
0.8912686479717378


**<span style="color:orange">Pipeline com RFE em um modelo de regressão XGB</span>**

Nesse pipeline foi aplicado apenas a técnica Recursive Feature Elimination.

In [172]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('recursiveFeatureElimination', RFE(RandomForestRegressor())),
    ('XGBRegressor', xgb.XGBRegressor())
])

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputerNumeric',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardScaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2nd...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [173]:
y_pred = model_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

924013701.4933233
0.8795339759308387


**<h3>Resultados encontrados</h3>**

Após aplicar os tratamentos de feature selection, não houve um incremento tão grande aplicando essas opções. Provalmente para esse caso não seja tão necessário, quando observamos apenas o erro quadrado médio e o R².

**<span style="color:orange">Erro quadrado médio</span>**
- **Sem nenhum tratamento adicional:** 842632962.4154353
- **Variância mínima:** 833833587.0615345
- **Correlação alta entre features:** 882726593.6872934
- **Variância e Correlação:** 834004939.0052686
- **RFE:** 957543952.2998086

**<span style="color:orange">R²</span>**
- **Sem nenhum tratamento adicional:** 0.8901
- **Variância mínima:** 0.8912
- **Correlação alta entre features:** 0.8849
- **Variância e Correlação:** 0.8912
- **RFE:** 0.8751